# Esquema para una Super-Plantilla DW (Ultra-bright)

## 1. Cargar librerías necesarias

In [1]:
# librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from pandas_profiling import ProfileReport
import os
from chardet.universaldetector import UniversalDetector


In [2]:

# ! pip install xlrd

In [3]:
# ajustes de visualización de Pandas:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 5000)

pd.set_option('display.precision', 2) #Cuando muestra decimales, lo hace hasta 2 decimas.
# pd.set_option('display.float_format', '{:.3f}'.format)

# por si nos hace falta matplotlib
%matplotlib inline

In [4]:
def whats_the_encoding_of(f:str, max_lines:int =-1) -> dict:
    '''
    Desc: Intenta adivinar cual es el encoding de f abriendo en binario con
          el límite de lineas propuesto en max_lines como argumento opcional.

    Nota: 1. Si el encoding es None o tarda mucho, conviene limitar las lineas a evaluar.
          2. Si el encoding de chardet es Windows-1254 (Turkish) se cambia manualmente
             a utf-8. Es un error de la librería cuando detecta emojis, independiente
             del texto que le acompañe.

    input  - f :str = Lugar, nombre y extension del archivo único a evaluar.
                      Lo abre en lectura binaria
           - (opcional) max_lines :int = Numero (positivo) máximo de lineas a evaluar.
                                         Por defecto es -1 para evaluar todas las lineas.

    output :dict {
        "encoding"    :str   = es el enconding detectado. Se reemplaza Windows-1254 (Turkish) por utf8.
        "confidence"  :float = la confianza en %,
        "language"    :str   = idioma tentativo. Vacio si no pudo identificarlo o si hay varios.
        "from"        :str   = direccion, nombre y extension del archivo evaluado.
        "forced_value":bool  = True si encuentra encoding en Windows-1254 (Turco) y confidece bajo .66.
                 }
    reference = https://stackoverflow.com/questions/61415196/python3-print-text-with-emojis-read-from-text-file-with-non-ascii-characters-u
    '''

    detector = UniversalDetector()
    for line in open(f, 'rb'):
        detector.feed(line)
        if (detector.done) or (max_lines == 0): break
        max_lines -= 1
    detector.close()
    detector.result['forced_value'] = False
    detector.result['from'] = f
    if (detector.result['encoding'] == 'Windows-1254') & (detector.result["confidence"] <= .66):
        detector.result['encoding'] = 'utf8' #Si hay error, intentar poner 'utf8sig', 'utf16' o 'utf32'
        detector.result['forced_value'] = True
    return detector.result

In [5]:

#MODIFICAD LA DIRECCIÓN AL ARCHIVO SEGÚN HAGA FALTA.
lugar_al_archivo = '.'
os.chdir(lugar_al_archivo)
cwd = os.getcwd()
cwd

'C:\\Users\\xhito\\Desktop\\DATA SCIENCE\\04 Mas Python por si acaso'

## 2. Cargar el dataset

In [10]:
df_train = pd.read_csv("./mascotas/train.csv")
#df_test = pd.read_csv("./carpeta/Mascotas/test.csv")

df_textos = pd.read_csv('./mascotas/mascotas_textos.csv')
df_entidades = pd.read_csv('./mascotas/mascotas_entidades.csv')
df_textos_entidades = pd.merge(df_textos, df_entidades, how ='inner', on = 'fichero', suffixes=['_textos','_merge'])
#quitamos el ".json" de fichero para poder usarlo como clave primaria.
df_textos_entidades['fichero'] = df_textos_entidades['fichero'].apply(lambda x: x[:-5])




In [11]:
#df1=suma de las columnas Vaccinated y Dewormed
df1=df_train.copy()
df1.insert(11, "Vac_Dew_1", df1["Vaccinated"]+df1["Dewormed"])
df1.drop(["Vaccinated","Dewormed"],axis=1, inplace=True)
df1.head(5)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vac_Dew_1,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,4,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,6,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,2,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,2,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,4,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [12]:
#df2=media de las columnas Vaccinated y Dewormed
df2=df_train.copy()
df2.insert(11, "Vac_Dew_2", (df2["Vaccinated"]+df2["Dewormed"])/2)
df2.drop(["Vaccinated","Dewormed"],axis=1, inplace=True)
df2.head(5)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vac_Dew_2,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,2.0,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3.0,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1.0,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,1.0,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,2.0,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [13]:
#df3. El valor 3 de las columnas Vaccinated, Dewormed y Sterilized pasa a valer 0 y después hacemos la media de las dos primeras columnas
df3=df_train.copy()
df3["Vaccinated"] = df3['Vaccinated'].apply(lambda x: 0 if x==3 else x) 
df3["Dewormed"] = df3['Dewormed'].apply(lambda x: 0 if x==3 else x) 
df3.insert(11, "Vac_Dew_3", (df3["Vaccinated"]+df3["Dewormed"])/2)
df3.drop(["Vaccinated","Dewormed"],axis=1, inplace=True)
df3.head(5)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vac_Dew_3,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,2.0,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,0.0,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1.0,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,1.0,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,2.0,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [14]:
#Añadir cada una de las tres columnas "Vac_Dew" al df_train

df_train.insert(11, "Vac_Dew_1", df1["Vac_Dew_1"])
df_train.insert(12, "Vac_Dew_2", df2["Vac_Dew_2"])
df_train.insert(13, "Vac_Dew_3", df3["Vac_Dew_3"])
df_train = pd.merge(df_train, df_textos_entidades, how ='inner', left_on = "PetID", right_on = 'fichero', suffixes=['_train','_t&e'])
df_train

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vac_Dew_1,Vac_Dew_2,Vac_Dew_3,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,fichero,texto,magnitud,score,nombre,tipo,salience,mention_content,mention_type
0,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,Nibble,ORGANIZATION,0.57,Nibble,PROPER
1,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,cuteness,OTHER,0.18,cuteness,COMMON
2,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,clinic,LOCATION,0.07,clinic,COMMON
3,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,cats,OTHER,0.03,cats,COMMON
4,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,result,OTHER,0.02,result,COMMON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541001,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,cages,OTHER,0.33,cages,COMMON
1541002,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,trees,OTHER,0.10,trees,COMMON
1541003,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,4,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,rooftops,OTHER,0.10,rooftops,COMMON
1541004,1,Fili,1,307,307,1,2,0,0,2,1,4,2.0,2.0,2,2,2,1,1,0,41332,9ed1d5493d223eaa5024c1a031dbc9c2,0,Fili just loves laying around and also loves b...,a83d95ead,1.0,3,a83d95ead,Fili just loves laying around and also loves b...,0.9,0.9,Fili,PERSON,0.73,Fili,PROPER


In [15]:
# eliminamos las columnas no deseadas de train y test
#columnas = ['Name','RescuerID','Description','PetID']
#df_train.drop(columnas,axis = 1,inplace=True)
#df_test.drop(columnas,axis=1,inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1541006 entries, 0 to 1541005
Data columns (total 36 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Type             1541006 non-null  int64  
 1   Name             1479182 non-null  object 
 2   Age              1541006 non-null  int64  
 3   Breed1           1541006 non-null  int64  
 4   Breed2           1541006 non-null  int64  
 5   Gender           1541006 non-null  int64  
 6   Color1           1541006 non-null  int64  
 7   Color2           1541006 non-null  int64  
 8   Color3           1541006 non-null  int64  
 9   MaturitySize     1541006 non-null  int64  
 10  FurLength        1541006 non-null  int64  
 11  Vac_Dew_1        1541006 non-null  int64  
 12  Vac_Dew_2        1541006 non-null  float64
 13  Vac_Dew_3        1541006 non-null  float64
 14  Vaccinated       1541006 non-null  int64  
 15  Dewormed         1541006 non-null  int64  
 16  Sterilized       1

In [16]:
#df_test.info()

NameError: name 'df_test' is not defined

### 2.1. Si falla la carga, utilizar chardet para la codificación correcta del dataset

### 2.2. En caso de tener muchos ficheros a cargar

## 3. Realizar el EDA automático

### 3.1. Pandas-Profiling

In [ ]:
# en caso de que el fichero se muy grande
# df = df.sample(int(len(df)/10))

### 3.2. DTale

### 3.3. Crear un EDA Personal

## 4. DW

### 4.1. Obtenemos Info() para saber qué tipo ha asignado a cada columna

#### 4.1.1. Si Pandas no reconoce correctamente el tipo de alguna columna, resolver.

#### 4.1.2. Eliminación de columnas irrelevantes o que no se podrá sacar información posteriormente

#### 4.1.3. Eliminación de filas duplicadas si es necesario y conveniente

### 4.2. Separamos columnas numéricas de categóricas en listas distintas. Separamos en otra lista las columnas temporales. Incluso las booleanas.

Para determinar si OneHot o LabelEncoding, buscar los valores únicos de las columnas categóricas y decidid: Maximo Label Encoding (hasta 5, siempre y cuando se "orienten" los valores por conocimiento del negocio)

#### 4.2.1 (Opcional) Eliminar todos los elementos no deseados, convertir a númerico (OneHot, LabelEncoding) y realizar un Feature Importances preeliminar

In [ ]:
#! pip install sklearn

In [17]:
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
# Feature importances
# prefijos = ['Nac', 'Club', 'P_Club', 'L_Club', 'Pos']

def F_importances(X,y):
    # Construye un arbol para así conocer la importancia de las columnas
    forest = ExtraTreesRegressor(n_estimators=250, random_state=0)
    columnas = X.columns.to_list()
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, columnas[indices[f]], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure(figsize=(10,10))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])


#F_importances(df_num.drop(["Value"],axis = 1),df_num['Value'])

#### 4.2.2 (Opcional) En caso contrario hacer una matriz de correlación con el target

### 4.3. Determinamos si tenemos pocos datos. En ese caso, buscaríamos nuevos conjuntos de datos (scrapping, otras fuentes) y los unimos (merge, concat) con los que tenemos (por filas o por columnas)

#### 4.3.1 Convertimos la columna Preferred Positions en una serie de columnas One hot

### 4.4. Extraemos información necesaria del EDA automático, DTale o del nuestro:
#### - Nulos por columna

#### - % valores únicos por filas y columnas

#### - Número de elementos por categoría en cada una de las columnas categóricas

#### - Filas duplicadas

#### - (Opcional) Eliminar todos los elementos no deseados, convertir a númerico (OneHot, LabelEncoding) y realizar un Feature Importances preeliminar

### 4.5 Limpieza de nulos

En este caso no aplica por no haber nulos

#### - Estrategia: Eliminación "con violencia" de filas o columnas con nulos

#### - Estrategia: rellenado valor fijo (con 0s, etc)

#### - Estrategia: media, mediana, moda (valor más frecuente en categóricos)

#### - Estrategia: valores extremos de la distribución

#### - Estrategia: valores aleatorios dentro del rango de valores de la columna

#### - Estrategia: ML

### 4.6 Análisis y corrección de distribuciones

##### Aquí podemos elegir la columna sobre la cual realizar el estudio, en cada una de las posibilidades se hagan 

##### Esto lo iremos repitiendo con todas las columnas numéricas y al final dejaremos esas columnas y no las originales cuando vayan al dataset

### 4.7 Tratamiento de outliers

#### 4.7.1. Detección de Outliers: Graficos Box-plot / Simetría y kurtosis

#### 4.7.2. Tratamiento si es necesario

### 4.8 Agrupamiento de categorias con pocas filas asociadas (categorias raras)

En este caso, no ha lugar a agrupar categorías

### 4.9 Correlaciones multivariable (numéricas)

### 4.10 Seleccion de características / reducción de dimensionalidad

### 4.11 Ingeniería de características

Este apartado está muy ligado a los datos, por lo que se rellenará con lo que se estime en cada dataset. Sólo se presentan ciertos apartados para ser trabajados si son necesarios.

#### - Fechas

#### - Crear columnas

#### - Agregaciones varias

#### - Label encoding dirigido a resultados

#### - Creación de columnas que indican existencia de nulos o valores raros

#### - Expansión polinómica (Estricta / No estricta)

#### - Eliminación de columnas definitivamente

### 4.12 Conversión categóricas a numéricas

#### - Label Encoding

##### - One Hot

### 4.13 Separar Target de Características

In [18]:
target ="AdoptionSpeed"

X_train = df_train.drop([target],axis = 1)
y = df_train[target]

In [19]:
X_train

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vac_Dew_1,Vac_Dew_2,Vac_Dew_3,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,fichero,texto,magnitud,score,nombre,tipo,salience,mention_content,mention_type
0,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,Nibble,ORGANIZATION,0.57,Nibble,PROPER
1,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,cuteness,OTHER,0.18,cuteness,COMMON
2,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,clinic,LOCATION,0.07,clinic,COMMON
3,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,cats,OTHER,0.03,cats,COMMON
4,2,Nibble,3,299,0,1,1,7,0,1,1,4,2.0,2.0,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,86e1089a3,Nibble is a 3+ month old ball of cuteness.,0.8,0.8,result,OTHER,0.02,result,COMMON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541001,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,cages,OTHER,0.33,cages,COMMON
1541002,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,trees,OTHER,0.10,trees,COMMON
1541003,2,Ms Daym,9,266,0,2,4,7,0,1,1,2,1.0,1.0,1,1,1,1,1,0,41336,c311c0c569245baa147d91fa4e351ae4,0,she is very shy..adventures and independent..s...,e4da1c9e4,3.0,e4da1c9e4,she is very shy..adventures and independent..s...,0.9,0.9,rooftops,OTHER,0.10,rooftops,COMMON
1541004,1,Fili,1,307,307,1,2,0,0,2,1,4,2.0,2.0,2,2,2,1,1,0,41332,9ed1d5493d223eaa5024c1a031dbc9c2,0,Fili just loves laying around and also loves b...,a83d95ead,1.0,a83d95ead,Fili just loves laying around and also loves b...,0.9,0.9,Fili,PERSON,0.73,Fili,PROPER


### 4.14 Volver a aplicar Correlaciones y Feature Importances (sin estandarizado)

### 4.15 Aplicar Estandarización

In [ ]:
def estandarizar(dataframe):
    media = dataframe.mean()
    desviacion_estandar = dataframe.std()
    return (dataframe - media)/desviacion_estandar

X = estandarizar(X_train)
X

### 4.16 (Opcional) Aplicar Feature Importances (con estandarizado)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
# Feature importances
# prefijos = ['Nac', 'Club', 'P_Club', 'L_Club', 'Pos']

def F_importances(X,y):
    # Construye un arbol para así conocer la importancia de las columnas
    forest = ExtraTreesRegressor(n_estimators=250, random_state=0)
    columnas = X.columns.to_list()
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %s (%f)" % (f + 1, columnas[indices[f]], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure(figsize=(10,10))
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])


#F_importances(df_num.drop(["Value"],axis = 1),df_num['Value'])

In [ ]:
F_importances(X,y)

# (Opcional) Un vistazo a cómo se comportaría el Dataset

In [ ]:
# cargamos algunas librerías de algoritmos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
# métricas clasificacion
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split


# cargamos algunas librerías de algoritmos de regresion
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
# métricas regresion
from sklearn.metrics import r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, # los datos (features) nuestra X
    y,  # columna objetivo, nuestra y
    stratify=y,
    test_size=0.1,  # el 90% de las filas serán para entrenar y el 10% para probar si ha ido bien
    random_state=0) 

X_train.shape, X_test.shape

In [ ]:
# clasificacion

# Aquí deshabilitamos el modelo que necesitemos
#modelo = LogisticRegression(random_state=44)
#modelo = AdaBoostClassifier(n_estimators=200)
modelo = RandomForestClassifier(n_estimators=200, n_jobs=-1)

# Entrenamos modelo
modelo.fit(X_train, y_train)

# predicción sobre el conjunto de test
y_pred = modelo.predict(X_test)

# calculamos su acierto
print('Accuracy: {}'.format(modelo.score(X_test, y_test)))
#print('roc-auc: {}'.format(roc_auc_score(y_test, y_pred)))